In [1]:
import xml.etree.ElementTree as ET
import os
import re

In [2]:
PATTERN1 = re.compile(r"(?<=[a-zA-Z])\n")
PATTERN2 = re.compile(r"\s+")

def clean_text(text):
    out = PATTERN1.sub(". ", text)
    return PATTERN2.sub(" ", out)

In [3]:
def load_data(path):
    data = []
    for file in os.listdir(path):
        with open(f"{path}/{file}", "r") as f:
            raw = f.read()
        root = ET.fromstring(raw)
        doc = clean_text(root.find("body").text)
        for summary in root.find("summaries").findall("summary"):
            if summary.attrib["ratio"] == "10":
                kind = summary.attrib["type"]
                text = clean_text(summary.find("body").text)
                data.append((doc, text, kind))
    return data

In [4]:
data = load_data("PSC_1.0/data")

# CLARIN-PL - Summarize

http://ws.clarin-pl.eu/summarize.shtml

http://nlp.pwr.wroc.pl/redmine/projects/nlprest2/wiki

http://nlp.pwr.wroc.pl/redmine/projects/nlprest2/wiki/Tools

In [5]:
docs = [data[0][0], data[-1][0]]

In [6]:
docs

['ŚWIĘTO NIEPODLEGŁOŚCI. Nic nie jest dobitniejszym świadectwem solidarności niż praca dla dobra wspólnego. Poszerzanie wolności. RYS. ANDRZEJ LEGUS. JERZY BUZEK. Święto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy. Jakie pożytki czerpiemy dziś z ponownie odzyskanej wolności? Jaka jest dzisiejsza wymowa słowa patriotyzm? Jak teraz układają się relacje między Polakami a ich państwem? Na czym w 1999 roku polega czyn obywatelski? Odpowiedzi na te pytania postanowiłem w tym roku poszukać w Krakowie, "mateczniku polskości", w mieście, gdzie historia i współczesność przenikają się w stopniu najpełniejszym. Dziedzictwo. 11 listopada to wielkie i radosne święto. Odzyskanie wolności zawsze stanowi tytuł do narodowej dumy. Dla Polaków czyn zbrojny, który przyniósł wolność, miał wymiar szczególny. 

In [7]:
import json
from urllib.request import urlopen, Request
import glob   
import os
import time
from rouge import Rouge

In [8]:
if not os.path.exists("summary"):
    os.mkdir("summary")

In [9]:
user = "moj@adres.email" 
task = 'any2txt|wcrft2({"morfeusz2":false})|liner2({"model":"names"})|summarize'
url = "http://ws.clarin-pl.eu/nlprest2/base"

out_path = 'summary/'

In [10]:
def upload(doc, url):
    return urlopen(Request(url + "/upload/", doc.encode(), {'Content-Type': 'binary/octet-stream'})).read()

In [11]:
def process(data, url):
    doc = json.dumps(data).encode('utf-8')
    taskid = urlopen(Request(url + '/startTask/', doc, {'Content-Type': 'application/json'})).read()
    time.sleep(0.2)
    resp = urlopen(Request(url + '/getStatus/' + taskid.decode("utf-8")))
    data = json.load(resp)
    while data["status"] == "QUEUE" or data["status"] == "PROCESSING":
        time.sleep(0.5)
        resp = urlopen(Request(url + '/getStatus/' + taskid.decode("utf-8")))
        data = json.load(resp)
    if data["status"]=="ERROR":
        print("Error " + data["value"])
        return None
    return data["value"]

In [12]:
global_time = time.time()
for idx, doc in enumerate(docs):
    doc_id = upload(doc, url)
    print(f"Processing: {idx}")
    data_ = {'lpmn' : task, 'user' : user, 'file' : doc_id.decode("utf-8")}
    data_ = process(data_, url)
    if data_ is None:
        continue
    data_ = data_[0]["fileID"]
    content = urlopen(Request(url + '/download' + data_)).read()
    with open(out_path + str(idx) + '.txt', "w") as f:
        f.write(content.decode("utf-8"))
print("GLOBAL %s seconds ---" % (time.time() - global_time))

Processing: 0
Processing: 1
GLOBAL 13.117507696151733 seconds ---


In [13]:
summaries = []
for i in range(len(docs)):
    with open(f"summary/{i}.txt", "r") as f:
        summaries.append(f.read())

In [14]:
print(summaries[0])

Nic nie jest dobitniejszym świadectwem solidarności niż praca dla dobra wspólnego.
Święto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy.
Odpowiedzi na te pytania postanowiłem w tym roku poszukać w Krakowie, "mateczniku polskości", w mieście, gdzie historia i współczesność przenikają się w stopniu najpełniejszym.
Ta z kolei stała się możliwa przede wszystkim dzięki żywotności polskiej tradycji i kultury.
One bowiem symbolizują łańcuch uzupełniających się działań zbrojnych i dyplomatycznych, cały czas wspieranych siłą polskiej kultury.
Tu wystarczy chwila skupienia, by uzmysłowić sobie, czym jest Polska, jakie dziedzictwo otrzymaliśmy w darze i jakie zobowiązania na nas ciążą.
Przekazywano ją w rodzinie, w Kościele, uczono z książek.
Dzięki tej pamięci naród przetrwał, a w kluczowym momenci

In [15]:
print(summaries[1])

Na zakup pomocy szkolnych wydaje się z budżetu oświaty niewiele pieniędzy - od 0,5 do 1 proc. - szacuje MEN.
Zazwyczaj pod koniec roku, kiedy nagle pojawiają się środki finansowe szkoły starają się je upłynnić i biorą wszystko, co się da, a producenci nie są w stanie sprostać zamówieniom.
Teraz wprawdzie funkcjonuje lista pomocy zalecanych przez MEN, ale nie jest ona obligatoryjna ani dla producentów, ani dla kupujących.
Skończył się też monopol MEN w zakresie produkcji i dystrybucji pomocy dydaktycznych.
Na rynku pomocy szkolnych trwa walka o klienta, walka między szkołokrążcami, oferującymi kiepski, a tani towar, którym nie zależy na umieszczeniu swoich produktów na liście MEN; dawnymi państwowymi producentami, przyzwyczajonymi do swej monopolistycznej pozycji oraz nowoczesnymi prywatnymi firmami, których produkty zbierają nagrody na krajowych i zagranicznych targach, jak np. ELBOX.
- Firmy państwowe, które działały na tym polu, były przyzwyczajone, że w MEN jest pewna pula pieniędzy

In [16]:
doc_hash1 = hash(data[0][0])
references1 = [s for d, s, _ in data if hash(d) == doc_hash1]

In [17]:
doc_hash2 = hash(data[-1][0])
references2 = [s for d, s, _ in data if hash(d) == doc_hash2]

In [18]:
rouge = Rouge()

In [19]:
summary1 = [summaries[0]] * len(references1)
summary2 = [summaries[1]] * len(references2)

In [20]:
scores1 = rouge.get_scores(summary1, references1, avg=True)
scores2 = rouge.get_scores(summary2, references2, avg=True)

In [21]:
for k, v in scores1.items():
    print(f"{k}: {v}")

rouge-1: {'f': 0.29654115271182935, 'p': 0.19441860465116279, 'r': 0.6267064196419978}
rouge-2: {'f': 0.20584424446663988, 'p': 0.13473193473193473, 'r': 0.43735206408434013}
rouge-l: {'f': 0.31405066720683344, 'p': 0.21493506493506492, 'r': 0.585829056166307}


In [22]:
for k, v in scores2.items():
    print(f"{k}: {v}")

rouge-1: {'f': 0.3279446590478781, 'p': 0.21426470588235294, 'r': 0.698931834557462}
rouge-2: {'f': 0.24517026553519652, 'p': 0.1600883652430044, 'r': 0.5235602426786029}
rouge-l: {'f': 0.34722453818885485, 'p': 0.23775280898876403, 'r': 0.6443945850535673}
